In [5]:
import time
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

2023-05-21 17:52:01.604426: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 17:52:01.643797: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 17:52:01.649912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 17:52:02.482322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [21]:

!ls

bad_example.jpeg	    good_example.jpeg	       ssd_mobilenet_v2
create_logo_detector.ipynb  model_main_tf2.py	       ssd_mobilenet_v2_ft
cropped-logo.jpg	    no_detection_example.jpeg  ssd_mobilenet_v2_ft2
detected-logo0.jpeg	    output		       testing.ipynb
exporter_main_v2.py	    _ssd_mobilenet_v2	       test.py


In [2]:
!python model_main_tf2.py --model_dir=output/ --pipeline_config_path=ssd_mobilenet_v2_ft/pipeline.config

2023-05-21 13:32:44.138422: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 13:32:44.203842: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-21 13:32:44.204146: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-21 13:32:45.274726: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/irizqy/ml_ws/bangkit-ws/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.


In [8]:
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ssd_mobilenet_v2/pipeline.config --trained_checkpoint_dir src/output/ --output_directory src/ssd_mobilenet_v2_ft2

2023-05-20 10:11:40.953422: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/irizqy/ml_ws/bangkit-ws/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-05-20 10:11:42.520505: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries fo

In [6]:
PATH_TO_MODEL_DIR = 'ssd_mobilenet_v2_ft/'
PATH_TO_LABELS = '/home/irizqy/ml_ws/bangkit-ws/data/label_map.pbtxt'

In [7]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "saved_model"

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...

2023-05-21 17:52:07.477413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-21 17:52:07.478249: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Done! Took 8.235520839691162 seconds


In [8]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [11]:
IMAGE_PATHS = ['/home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/36_20230510_162147.jpg',
               '/home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/12_IMG20230504165724.jpg',
               '/home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/20_20230510_161931.jpg',]

In [12]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings
count = 0

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = load_image_into_numpy_array(image_path)
    
    if image_np.shape[2] == 4:
        image_np = image_np[:, :, :3]

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)
    # print(detections)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure()
    plt.imshow(image_np_with_detections)
    im = Image.fromarray(image_np_with_detections)
    im.save(f"detected-logo2{count}.jpeg")
    count+=1
    print('Done')
    plt.show()

# sphinx_gallery_thumbnail_number = 2

Running inference for /home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/36_20230510_162147.jpg... defaultdict(<class 'str'>, {(0.2317146360874176, 0.3359919786453247, 0.3352116644382477, 0.7213940620422363): 'Aqua'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/12_IMG20230504165724.jpg... defaultdict(<class 'str'>, {(0.27207693457603455, 0.11005356907844543, 0.37995490431785583, 0.911182165145874): 'Aqua', (0.26456883549690247, 0.11640459299087524, 0.35175302624702454, 0.6515470147132874): 'Aqua'})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/20_20230510_161931.jpg... defaultdict(<class 'str'>, {})
Done
Running inference for /home/irizqy/ml_ws/bangkit-ws/data/bizit-dev_test-data/40_20230507_094336.jpg... defaultdict(<class 'str'>, {(0.3223600387573242, 0.425650954246521, 0.431532084941864, 0.828102707862854): 'Aqua'})
Done


In [9]:
import cv2 as cv

In [131]:
# print(detections['detection_scores'])
# print(detections['detection_boxes'])
# print(detections['detection_classes'])
detections.items()

dict_items([('detection_anchor_indices', array([1467.,  447.,  490.,  508.,  880.,  514.,  577.,  448.,  880.,
        362.,  444.,  507.,  451.,  583.,  579.,  876.,  459.,  519.,
        520.,  494.,  568.,  582.,  879.,  504., 1841.,  564., 1935.,
        885., 1869.,  564., 1975.,  819.,  522.,  882.,  585.,  360.,
       2019.,  420.,  567.,  639.,  462.,  556.,  642., 2005.,  489.,
       1767., 2025.,  492., 1972., 1993.,  816.,  879., 2028.,  822.,
        414., 1131., 1134.,  561., 1467.,  585.,  821.,  540., 1905.,
        480.,  460.,  474., 1957.,  362.,  856., 2013., 1862.,  490.,
       2016., 1132., 1444., 1076., 1074.,  495.,  852., 1131.,  825.,
        240., 1630., 1839.,  305., 1134., 1477.,  819.,  576.,  882.,
       1132., 2031., 2006., 2033.,  243.,  789.,  549., 2011., 1959.,
        861.], dtype=float32)), ('detection_multiclass_scores', array([[0.00931161, 0.04947763, 0.6265359 ],
       [0.00674042, 0.02613527, 0.2936889 ],
       [0.00840167, 0.25192142, 0.0

In [10]:
width, height, _ = image_np.shape

raw_detection_boxes <br>
detection_scores<br>
detection_boxes<br>
detection_anchor_indices<br>
raw_detection_scores<br>
detection_classes<br>
detection_multiclass_scores<br>
num_detections

In [11]:
ymin, xmin, ymax, xmax = detections['detection_boxes'][0]
(left, right, top, bottom) = (xmin*width, xmax*width, ymin*height, ymax*height)
print(int(left), int(right), int(top), int(bottom))

25 265 118 171


In [134]:
im_top = cv.circle(image_np, (int(top), int(top+right)), 2, (0, 266, 0), 2)
# im_bot = 
# im_right = 
# im_lrft = 

Image.fromarray(im_top).save('top.jpg')

In [135]:
im = cv.rectangle(image_np, (int(left), int(top)), (int(right), int(bottom)), (0, 255, 0), 3)
Image.fromarray(im).save('test.jpg')

In [13]:
cropped_im = image_np[int(top):int(bottom), int(left):int(right)]
Image.fromarray(cropped_im).save('cropped-logo.jpg')
